WalkSAT plus Data Analysis of Results, Daniel Espinosa 2024 UCSB ECE Strukov Group + FPGA copy


![alt text](firefox_FtRzgqGGfO.png)[size:2]

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator
import xml.etree.ElementTree as Xet

In [12]:
import random

def walkSAT(clauses, max_tries, max_flips, p):
    def evaluate_clause(clause, assignment):
        return any((var > 0 and assignment.get(abs(var), False)) or 
                   (var < 0 and not assignment.get(abs(var), False)) for var in clause)

    def get_unsatisfied_clauses(clauses, assignment):
        return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

    def get_variables(clauses):
        return set(abs(var) for clause in clauses for var in clause)

    def flip_variable(assignment, var):
        assignment[var] = not assignment[var]

    for _ in range(max_tries):
        variables = list(get_variables(clauses))
        assignment = {var: random.choice([True, False]) for var in variables}
        
        for _ in range(max_flips):
            unsatisfied = get_unsatisfied_clauses(clauses, assignment)
            if not unsatisfied:
                return assignment  # Found a satisfying assignment
            
            clause = random.choice(unsatisfied)
            if random.random() < p:
                # Flip a random variable from the clause
                var_to_flip = abs(random.choice(clause))
            else:
                # Flip a variable that minimizes the number of unsatisfied clauses if flipped
                break_counts = []
                for var in clause:
                    assignment[abs(var)] = not assignment[abs(var)]
                    break_counts.append((len(get_unsatisfied_clauses(clauses, assignment)), abs(var)))
                    assignment[abs(var)] = not assignment[abs(var)]  # Undo the flip

                min_break = min(break_counts, key=lambda x: x[0])
                vars_with_min_break = [var for break_count, var in break_counts if break_count == min_break[0]]
                var_to_flip = random.choice(vars_with_min_break)
            
            flip_variable(assignment, var_to_flip)

    return "FAIL"

# Example SAT CNF formula: (x1 or not x2) and (not x1 or x2)
clauses = [[1, -2],[-1,2]]
# Example UNSAT CNF formula: 
clauses = [[1, 2],[-1,-2],[1,-2],[-1,2]]
# Parameters
max_tries = 100
max_flips = 1000
p = 0.3

# Running WalkSAT
result = walkSAT(clauses, max_tries, max_flips, p)
print("Satisfying assignment found:" if result != "FAIL" else "No satisfying assignment found within the given limits.", result)

No satisfying assignment found within the given limits. FAIL
